# Large Model Memory 효율적 사용하는 방법 (huggingface 기준)
* 최근 나오는 LLM 모델들의 경우 메모리 사용량이 많은데 메모리 사용을 효율적으로 사용 할 수 있는 방법들 기록
* 항상 성능과 효율의 적정선을 찾기 위해 노력해야함

1. Model load 시에 8bit 로 로딩
  * AutoModelForCausalLM.from_pretrained() 의 옵션으로 load_in_8bit=True
  * 모델의 가중치와 활성화 함수의 출력이 8 bit 정밀도로 저장되어 처리
    * 활성화 함수의 출력이란? => ReLU, Sigmoid 등의 활성화 함수가 입력 신호를 받아 처리한 결과
  * 기존 학습된 32 bit(float) 형식에 비해 8 bit 정밀도로 로드시 메모리 사용량이 감소 => 데이터 전송 속도가 빨라지고 연산이 더 효율적으로 이뤄져 모델 훈련과 추론 속도 향상
  * 32 bit(float) 형식에 비해 8 bit 의 정밀도가 낮아 모델의 정확도와 같은 성능은 떨어질 수 있음
  * 하드웨어가 8bit 연산을 지원하는지 확인하고 사용해야함

2. 모델 훈련 전 모델을 얼리기(freeze)
  * param.requires_grad = False 를 이용
  * 모델의 모든 파라미터에 대해 역전파 과정에서 그라디언트 계산을 수행하지 않도록 설정
  * 이는 추후 어댑터(adapter)를 훈련할 때를 제외하고는 모델의 가중치가 업데이트되지 않도록 하는 데 사용

3. 모델 훈련시 Gradient Checkpointing 적용
  * model.gradient_checkpointing_enable() 함수를 이용
  * 메모리 집약적 대규모 모델 훈련시 메모리 효율성을 크게 향상
  * 기존 모델의 학습시에는 backpropagation 에서 gradient 계산을 하기 위해 신경망은 forward pass 동안의 각 레이어 출력인 activations를 저장하는데 이를 모두 저장하는것은 메모리를 많이 소비
  * 모든 activations를 저장하는 대신 특정 레이어에서만 activations 를 저장하고 필요시 나머지를 다시 계산하여 전체적인 메모리 사용량을 줄임
  * 이를 이용하면 메모리 절약으로 더 큰 배치크기로 훈련이 가능하거나 GPU 메모리 제약으로 Train 이 힘든 LM 도 훈련 가능
    * 더 큰 배치크기로 훈련하면 한 번의 업데이트에 더 많은 데이터를 사용하여 학습 과정에서 더 안정적인 경사 하강을 가능하게 하여 안정성을 가져옴
  * 하지만 activations 를 다시 계산해야하니 계산시간 증가로 훈련 시간은 증가할 수 있음

4. LoRA(Low-Rank Adaptaion) / QLoRA(Quantized Low-Rank Adaptation)와 같은 PEFT 사용
  * LoraConfig() 사용
  * LoRA 방법은 모델의 기존 가중치는 고정한 뒤 저차원 행렬을 도입하여 기존 가중치에 적용 => full fine tuning 에 비해 시간 절약
  * 전체 모델의 모든 가중치를 미세 조정하는 대신, 작은 수의 추가 파라미터를 사용하여 모델의 일부분만을 조정
  * config 에 설정한 하이퍼 파라미터에 의존 (복잡도 조정 가능 => 모델 용량과 성능 균형조절 가능)

5. 모델 훈련시 fp16 사용
  * transformers.TrainingArguments() 옵션에 fp16=True 설정
  * 훈련시 16비트 부동소수점(Half Precision)을 사용
  * 메모리 사용량을 줄임 => FP16 은 FP32 부동소수점(Full Precision)에 비해 메모리 사용량 절만 사용
  * 훈련 / 추론 속도를 높임 => 메모리의 대역폭이 감소하여 연산 속도가 빨라짐
  * FP32 에 비해 FP16 은 낮은 정밀도로 정확도에 영향을 줄 수 있음 => FP16 과 FP32 를 혼합하여 사용 (Mixed Precision)
  * 하드웨어가 FP16 연산을 지원하는지 확인하고 사용해야함
6. 모델 훈련시 Gradient Accumulation 사용
  * transformers.TrainingArguments() 옵션에 gradient_accumulation_steps 사용
  * 모델 훈련 시 배치에 대한 gradient 를 즉시 업데이트 하지 않고, 여러 스탭에 걸쳐 누적된 gradient 를 업데이트
  * 더 큰 배치 크기를 학습시킨 효과를 냄 => 큰 배치 크기는 gradient update의 분산을 줄임 => 안정적인 최적화
  * 각 스탭에서 처리하는 실제 데이터 양이 작게 유지되어 GPU 메모리 사용량 감소

7. DeepSpeed / Acceralate
  *  Accelerate는 자동으로 데이터 병렬 처리, 모델 병렬 처리를 관리하며, 간단한 설정을 통해 여러 GPU 또는 TPU에서 효율적으로 모델을 훈련할 수 있도록 지원
  * DeepSpeed는 마이크로소프트에서 개발한 라이브러리 / 제로 Redundancy Optimizer(ZeRO), 모델 병렬 처리, 그리고 그라디언트 체크포인팅과 같은 기술을 통해 메모리 사용량을 현저히 줄임
    * 작은 메모리를 가진 하드웨어에서도 대규모 모델을 훈련
    * ZeRO는 모델의 파라미터, 그라디언트, 옵티마이저 상태를 효율적으로 관리하여, 기존 방식 대비 메모리 사용량을 크게 줄이면서도 훈련 속도를 향상
  * 위 두 내용은 아래에서 추가 기술
    * 주소

## 관련 라이브러리 설치

In [1]:
!pip install -q datasets==2.16.1
!pip install -q transformers==4.36.2

!pip install bitsandbytes==0.42.0
!pip install peft==0.7.1
!pip install accelerate==0.26.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [2]:
!nvidia-smi

Thu Mar  7 04:54:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Model load 시 8bit 로 로딩

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_int8_training

model_id = 'google-bert/bert-base-cased'
tokenizer_id = model_id

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    trust_remote_code=True
)

model = prepare_model_for_int8_training(model)

tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

0

## 모델 훈련 전 모델을 얼리기(freeze)

In [5]:
import torch

for param in model.parameters():
  # 모델의 모든 파라미터에 대해 역전파 과정에서 그라디언트 계산을 수행하지 않도록 설정
  param.requires_grad = False
  # 모델 내의 차원이 1인 파라미터 =>  파라미터는 상대적으로 작기 때문에 FP32로 변환하여 계산의 안정성을 높임
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

# 부분 얼리기
# 특정 레이어 이름을 알면
for name, param in model.named_parameters():
    if 'encoder.layer.0' in name:
        param.requires_grad = False

## 모델 훈련시 Gradient Checkpointing 적용
* 훈련 중에 저장되는 활성화(activation)의 수를 줄여 메모리 사용량을 감소시키지만, 약간의 계산 오버헤드를 추가

In [6]:
model.gradient_checkpointing_enable()

In [7]:
### 모델 트레인 가능 파라미터 확인 (얼마나 얼려졌는지)
def check_trainable_parameters(model):
  trainable_parameters = 0
  all_parameter = 0
  for param in model.parameters():
    # param.numel(): 해당 파라미터 텐서에 포함된 요소의 총 수를 반환
    all_parameter += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  # {: ,} , 는 숫자를 1000단위로 구분
  print(f"all_parameter: {all_parameter:,} / trainable_parameters: {trainable_parameters:,}")
check_trainable_parameters(model)

all_parameter: 108,369,800 / trainable_parameters: 0


## LoRA(Low-Rank Adaptaion) / QLoRA(Quantized Low-Rank Adaptation)와 같은 PEFT 사용

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, # LoRA 어댑터에 의해 추가되는 파라미터의 수
    lora_alpha=32, # LoRA 어댑터가 입력에 미치는 영향의 정도를 결정
    lora_dropout=0.05, # 모델이 과적합되는 것을 방지
    bias="none", # 편향을 추가하는 것은 모델의 표현력을 높일 수 있지만, 동시에 계산 비용을 증가
    task_type="CAUSAL_LM" # 인과적 언어 모델링(Causal Language Modeling)
    )

# QLoRA
# from peft import LoftQConfig
# LoftQConfig(loftq_bits=4)
# lora_config = LoraConfig(
#     init_lora_weights="loftq",
#     loftq_config=loftq_config,
#     r=16,
#     lora_alpha=8,
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

model = get_peft_model(model, config)


### train 을 위한 임시 sample dataset
* 실제로는 다른 데이터셋 필요

In [9]:
# 학습을 위한 데이터 로드
from datasets import load_dataset
tokenizer.pad_token = tokenizer.eos_token

ori_dataset = load_dataset("beomi/KoAlpaca-v1.1a")
ori_dataset


Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [10]:
from datasets import DatasetDict
dataset = DatasetDict({ 'train': ori_dataset['train'].select(range(100))})
dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 100
    })
})

In [12]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = dataset.map(
    lambda x: tokenizer(x['output'], padding='max_length', truncation=True, max_length=512),
    batched=True
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## 모델 훈련시 fp16 사용 & Gradient Accumulation


In [13]:
import transformers
train_args=transformers.TrainingArguments(
    output_dir='./result',
    per_device_train_batch_size=2,
    gradient_accumulation_steps=6,
    fp16=True
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [14]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


TrainOutput(global_step=24, training_loss=4.375852902730306, metrics={'train_runtime': 53.3751, 'train_samples_per_second': 5.621, 'train_steps_per_second': 0.45, 'total_flos': 76349128310784.0, 'train_loss': 4.375852902730306, 'epoch': 2.88})

### 학습 모델 확인


In [15]:
pretrained_model_path = 'memorty_test_model'  # it will be directory
model.save_pretrained(pretrained_model_path)

In [16]:
# 불러온 모델 peft 적용
lora_config = LoraConfig.from_pretrained(pretrained_model_path)
model = get_peft_model(model, lora_config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): BertLMHeadModel(
          (bert): BertModel(
            (embeddings): BertEmbeddings(
              (word_embeddings): Embedding(28996, 768, padding_idx=0)
              (position_embeddings): Embedding(512, 768)
              (token_type_embeddings): Embedding(2, 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (encoder): BertEncoder(
              (layer): ModuleList(
                (0-11): 12 x BertLayer(
                  (attention): BertAttention(
                    (self): BertSelfAttention(
                      (query): lora.Linear8bitLt(
                        (base_layer): Linear8bitLt(in_features=768, out_features=768, bias=True)
                        (lora_dropout): ModuleDict(
                          (default): Dropou

In [17]:
input = tokenizer("양파는 ", return_tensors='pt')
# PyTorch에서 제공하는 자동 혼합 정밀도(Automatic Mixed Precision, AMP) 기능을 사용하는 컨텍스트 매니저
# 연산을 더 빠르고 메모리 효율적으로 수행할 수 있도록 도와줌
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**input, max_new_tokens=50)

tokenizer.decode(output_tokens[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'..................................................................................'